In [1]:
from utils import *
import pandas as pd

In [2]:
df_mme = pd.read_csv("data/M.csv")

In [3]:
df_mme.query("Estado != 'R'")

,Identificador,Fecha,Estado,Tipo,Cuenta,Beneficiario,Categoría,Importe,Divisa,Número,Notas
1448,2544,20/12/2022,NaN,Withdrawal,OPENBANK ECUENTA,Desconocido,Desconocido,-20.0,EUR,NaN,NaN


In [4]:
df = df_mme.pipe(clean_data).query("Fecha >= '2022-01-01'")
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [5]:
df.sample(5)

,Fecha,Tipo,Categoria,Subcategoria,Beneficiario,Transaccion,Importe
1218,2022-07-27,Ingresos,Despacho - Ingresos,Ana,SILVIA GARCIA ROLDAN,Abono,400.0
1831,2022-08-04,Gastos,Extras,Viajes,BUDAPEST,Cargo,-35.29
1839,2022-08-08,Gastos,Extras,Viajes,BUDAPEST,Cargo,-8.96
2055,2022-11-07,Gastos,Extras,Varios,BIOCULTURA,Cargo,-13.48
808,2022-07-07,Gastos,Extras,Varios,AMAZON,Cargo,-159.9


In [7]:
(df_hogar
    .pipe(pivot_by_category_totals, ["Tipo"])
        #.pipe(style_locale_es)
       .pipe(style_dataframe_totals, ['Tipo'])
 )

,Tipo,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,"-9.125,04","-8.505,44","-12.485,53","-5.234,44","-3.461,10","-8.288,79","-7.771,75","-5.210,00","-5.021,79","-5.208,63","-4.370,48","-4.858,84","-6.628,49","-79.541,83"
1,Ingresos,"11.082,86","10.296,45","9.266,42","2.727,39","2.587,06","5.291,84","9.110,07","5.627,59","5.602,14","7.735,52","9.696,62","2.642,34","6.805,52","81.666,30"
2,Total,"1.957,82","1.791,01","-3.219,11","-2.507,05","-874,04","-2.996,95","1.338,32","417,59","580,35","2.526,89","5.326,14","-2.216,50","177,03","2.124,47"


In [19]:
(df_despacho
     .pipe(pivot_by_category_totals, ["Tipo"])
     .pipe(style_locale_es)
 )

,Tipo,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,"-263,54","-490,46","-2.310,61","-440,76","-283,38","-1.430,33","-431,15","-593,03","-1.782,86","-638,43","-655,18","-627,95","-828,97","-9.947,68"
1,Ingresos,"1.938,61","4.446,89","8.752,27","242,00","50,00","3.903,71","3.670,00","3.413,30","4.578,71","5.334,69","3.290,60","410,00","3.335,90","40.030,78"
2,Total,"1.675,07","3.956,43","6.441,66","-198,76","-233,38","2.473,38","3.238,85","2.820,27","2.795,85","4.696,26","2.635,42","-217,95","2.506,93","30.083,10"
